In [1]:
%matplotlib inline
import pandas as pd
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn import cross_validation as cv
from sklearn import preprocessing
from sklearn import ensemble as en
from sklearn import cross_validation
import matplotlib.pyplot as plt
import seaborn as sns

import wikipedia
import pandas as pd
import csv
import re

__File containing Athlete names__

In [ ]:
fopen = open("C:\\Users\\padhu\\Documents\\Spring 2016\\MSA 8150 Machine Learning\\Final project\\Cleaned files\\names1.csv",mode='r')
names_list = fopen.readlines()
names_list = [item.strip('\n') for item in names_list]
names_list = list(set(names_list))

name_gender = {}
name_age= {}

__Determining Gender from wikipedia page__

In [ ]:
male_words = [' his ',' him ',' he ']
female_words = [' her ',' she ']

def getGender(name):
    try:
        t= wikipedia.page(name)
        summary = t.summary
        summary = summary.replace("\n"," ")
        summary = summary.lower()
        if ( (summary.find(female_words[0]) >0 or summary.find(female_words[1]) >0) and (summary.find(male_words[0]) >0 or  summary.find(male_words[1]) >0 or summary.find(male_words[2])>0)):
            return "found both"
        elif (summary.find(female_words[0]) >0 or summary.find(female_words[1]) >0):
            return "female"
        elif (summary.find(male_words[0]) >0 or  summary.find(male_words[1]) >0 or summary.find(male_words[2])>0):
            return "male"
        else:
            return "gender not found"
        
    except:
        return "gender not found"
    
for i in range(len(names_list)):
    name_gender[names_list[i]] = getGender(names_list[i])



__Extract Year of birth from Wikipedia page__

In [ ]:
yob = {}
for i in range(len(names_list)):
    try:
        t= wikipedia.page(names_list[i])
        summary = t.summary
        summary = summary.lower()
        summary = summary.replace("\n","") 
        yob[names_list[i]] = re.search('\((.*?)\)',summary).group(1)
        
    except:
        yob[names_list[i]] = "yob not found"

__Input Data and Preprocessing__

In [2]:
df = pd.DataFrame.from_csv("C:\\Users\\padhu\\Documents\\Spring 2016\\MSA 8150 Machine Learning\\Final project\\datafile.csv",index_col = None)
df1 = df.drop(['Age','Name','Country'],axis=1)
#df1

le = preprocessing.LabelEncoder()
for item in list(df1.columns):
    temp = le.fit(list(df1[item]))
    t = le.transform(list(df1[item]))
    df1[item] = le.transform(list(df1[item]))

df_test = df1.sample(30)

X= df1.drop('Doping Indicator',axis=1)
y = df1['Doping Indicator']
#df1.columns

__Split the data for Training and Testing__

In [3]:
X_train, X_test, y_train, y_test = cv.train_test_split(X, y, test_size=0.20, random_state=42)

df_train = pd.concat([X_train,y_train],axis=1)

__Naive Bayes model- Training__

In [17]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

__Naive Bayes- Prediction__

In [18]:
mnb.predict_proba(X_test)
X_train.columns

Index([u'Gender', u'Sport', u'Medal', u'Age_group', u'Country_Medal_group'], dtype='object')

__Decison Tree__

In [45]:
from sklearn import tree

cols = df1.columns

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)


from sklearn.externals.six import StringIO  
import pydot 
from IPython.display import Image  

dot_data = StringIO()
tree.export_graphviz(clf, out_file=dot_data,
    feature_names=cols,filled=True) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("doping.pdf") 
Image(graph.create_png())  

ValueError: Number of labels=80 does not match number of samples=7008

__Balanced Random Forest sampling__

2313

In [5]:
minor_class_df = df_train[df_train['Doping Indicator'] == 1]
major_class_df = df_train[df_train['Doping Indicator'] == 0]

t = pd.concat([major_class_df.sample(15),minor_class_df.sample(15)])
#t= minor_class_df[:5]
xt = t.drop('Doping Indicator',axis=1)
yt = t['Doping Indicator']

#xt = X_test
#yt = y_test

scores = []
scores_list={}
predict = []

for i in range(100):
    temp_df = pd.concat([major_class_df.sample(40),minor_class_df.sample(40)])
    x = temp_df.drop('Doping Indicator',axis=1)
    y = temp_df['Doping Indicator']
    mod = en.RandomForestClassifier(n_estimators=10, oob_score = True)
    mod.fit(x,y)
    print "actual:", sum(yt),"predicted:",sum(mod.predict(xt))
    predict.append(mod.predict(xt))
    score = cross_validation.cross_val_score(mod,x,y,cv=10)
    #print sum(score)/len(score)
    scores.append(sum(score)/len(score))
    scores_list[i]=score

actual: 15 predicted: 18
actual: 15 predicted: 19
actual: 15 predicted: 21
actual: 15 predicted: 18
actual: 15 predicted: 17
actual: 15 predicted: 18
actual: 15 predicted: 14
actual: 15 predicted: 18
actual: 15 predicted: 17
actual: 15 predicted: 17
actual: 15 predicted: 20
actual: 15 predicted: 15
actual: 15 predicted: 18
actual: 15 predicted: 17
actual: 15 predicted: 20
actual: 15 predicted: 20
actual: 15 predicted: 18
actual: 15 predicted: 18
actual: 15 predicted: 19
actual: 15 predicted: 14
actual: 15 predicted: 17
actual: 15 predicted: 18
actual: 15 predicted: 16
actual: 15 predicted: 19
actual: 15 predicted: 19
actual: 15 predicted: 17
actual: 15 predicted: 15
actual: 15 predicted: 17
actual: 15 predicted: 18
actual: 15 predicted: 16
actual: 15 predicted: 19
actual: 15 predicted: 17
actual: 15 predicted: 21
actual: 15 predicted: 19
actual: 15 predicted: 21
actual: 15 predicted: 17
actual: 15 predicted: 17
actual: 15 predicted: 21
actual: 15 predicted: 14
actual: 15 predicted: 20


In [79]:
#plt.hist(predict)

In [66]:
predict_count = [sum(item) for item in predict]
#predict_count
scores
type(scores_list)

dict

In [139]:
Amanda = pd.concat([X_test,y_test],axis=1)
Amanda.reset_index(inplace=True)
Myanka = pd.DataFrame(predict[5],columns = ['predicted'])


final = pd.concat([Amanda,Myanka],axis=1)

#df_train
#pd.concat([X_test,y_test],axis=1)
#pd.DataFrame(predict[5],index= None)


final

final.columns

cols = ['Gender', 'Sport', 'Medal', 'Age_group','Country_Medal_group']

for item in cols:
    


Index([u'index', u'Gender', u'Sport', u'Medal', u'Age_group',
       u'Country_Medal_group', u'Doping Indicator', u'predicted'],
      dtype='object')